# Ben's

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[(world.pop_est>0) & (world.name!="Antarctica")]
world['gdp_per_cap'] = world.gdp_md_est / world.pop_est
fig, ax = plt.subplots(nrows=1, ncols=1,figsize = (25,12))
df_starbucks=gpd.GeoDataFrame(df_starbucks)
base=world.plot(ax=ax,column='gdp_per_cap', cmap='OrRd',legend='on');
df_starbucks.plot(ax=base,marker='o',color='r',markersize=3)
plt.axis('off')
plt.title("Starbucks distribution under population density")
plt.show()

# above uses gdp per cap, which does not make sense. I suggest we change to this:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world[(world.pop_est>0) & (world.name!="Antarctica")]
fig, ax = plt.subplots(nrows=1, ncols=1,figsize = (25,12))
df_starbucks=gpd.GeoDataFrame(df_starbucks)
base=world.plot(ax=ax,column='pop_est', cmap='OrRd',legend='on');
df_starbucks.plot(ax=base,marker='o',color='b',markersize=3)
plt.axis('off')
plt.title("Starbucks distribution under population density")
plt.show()

In [ ]:
#check for null values
print("There are totally %s rows containing NaNs."%sum(df_profile.isnull().any(axis=1)))

In [ ]:
#Since the number of missing zip codes are relatively small to the total amount, and the number of people living in this 
#area are quite small, so we can drop them due to this consideration.
neg=(df_census['total_population']<0) | (df_census['median_age']<0) | (df_census['median_hh_income']<0) | (df_census['median_home_value']<0| (df_census['median_rent']<0) )
num=neg.sum()
print("There are totally %s columns containing negative numbers."%num)

In [ ]:
#According to the official guideline which says that negative very big numbers indicates non-existent data, so we need to 
#drop them.
pos=(df_census['total_population']>0) & (df_census['median_age']>0) & (df_census['median_hh_income']>0) & (df_census['median_home_value']>0)& (df_census['median_rent']>0)
df_census=df_census[pos]
df_census

In [ ]:
pd.options.display.max_columns=100

In [ ]:
df_census.rename({'zip code tabulation area':'zip'},axis='columns',inplace=True)
df = pd.merge(df_census, df_US, how='left', on=['zip'])
pd.options.display.max_columns=100
df

In [ ]:
sns.set_style('whitegrid')
ax=sns.boxplot(x='has_starbucks',y='median_hh_income',hue='has_starbucks',data=df_census,palette='Set3')

In [ ]:
ax=sns.boxplot(x='has_starbucks',y='median_age',hue='has_starbucks',data=df_census,palette='Set3')

In [ ]:
ax=sns.boxplot(x='has_starbucks',y='percent_bachelor',hue='has_starbucks',data=df_census,palette='Set3')

In [ ]:
ax=sns.boxplot(x='has_starbucks',y='median_rent',hue='has_starbucks',data=df_census,palette='Set3')

In [ ]:
ax=sns.boxplot(x='has_starbucks',y='median_home_value',hue='has_starbucks',data=df_census,palette='Set3')

# Ryan's

In [ ]:
# TODO: Kim, The next 3 cells are mine, use them to replace the previous code (for-loop with has_starbucks setting)

# Quicksorting data
df_census = df_census.sort_values(['zip code tabulation area'],kind="quicksort")

df_US = df_US.sort_values(['zip'],kind="quicksort")
df_US

In [ ]:
#Progress Bar for fun
import time,sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 20
    if (isinstance(progress,int)):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length*progress))
    clear_output(wait=True)
    text = "Progress: [{0}] {1:.1f}%" .format( ("#" * block) + ("-" * (bar_length - block)), progress * 100)
    print(text)

In [ ]:
#Find all zip codes with a starbucks - this will take some time (approx. 13 min) so I added a progress bar
df_census['has_starbucks'] = 0
df_census['starbucks_amount'] = 0
#length = len(df_census.index)
#count = 0
#prog = 0
for index, row in df_census.iterrows():    
    #count = count + 1
    #prev_prog = prog
    #prog = count/length
    #if ( prog != prev_prog ):
    #    update_progress(prog)
    #print("Currently Working on " + str(row['zip code tabulation area']))
    has_sb = df_US['zip'].searchsorted(row['zip code tabulation area'])
    if (has_sb > 0):
        has_sb = 1
        num_sb = df_US.zip.str.count(row['zip code tabulation area']).sum()
        df_census.loc[index,['has_starbucks','starbucks_amount']] = [has_sb,num_sb]
    
df_census

# Kim's

In [ ]:
#get count of starbucks in zipcode
num_starbucks_in_zipcode = df_starbucks_US.pivot_table(index=['zip'], aggfunc = 'size')
num_starbucks_in_zipcode = pd.DataFrame({'zip':num_starbucks_in_zipcode.index, 'starbucks count':num_starbucks_in_zipcode.values})
num_starbucks_in_zipcode

In [ ]:
df_US = df_starbucks_US.merge(df_census, how='right', on='zip')
#fill NaNs with 0s